## 🚗 Car Manufacturer Suggested Retail Price (MSRP) Prediction

Given *data about various cars*, let's try to predict the **suggested retail price** of a given car.

We will be using a variety of regression models to make our predictions.

Data source: https://www.kaggle.com/datasets/CooperUnion/cardataset

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import LinearSVR, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [2]:
data = pd.read_csv('data.csv')
data

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

### Preprocessing

In [4]:
df = data.copy()

In [5]:
{column: len(df[column].unique()) for column in df.select_dtypes('object').columns}

{'Make': 48,
 'Model': 915,
 'Engine Fuel Type': 11,
 'Transmission Type': 5,
 'Driven_Wheels': 4,
 'Market Category': 72,
 'Vehicle Size': 3,
 'Vehicle Style': 16}

In [6]:
df['Vehicle Size'].unique()

array(['Compact', 'Midsize', 'Large'], dtype=object)

In [7]:
df.select_dtypes('object')

,Make,Model,Engine Fuel Type,Transmission Type,Driven_Wheels,Market Category,Vehicle Size,Vehicle Style
0,BMW,1 Series M,premium unleaded (required),MANUAL,rear wheel drive,"Factory Tuner,Luxury,High-Performance",Compact,Coupe
1,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Convertible
2,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,High-Performance",Compact,Coupe
3,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,"Luxury,Performance",Compact,Coupe
4,BMW,1 Series,premium unleaded (required),MANUAL,rear wheel drive,Luxury,Compact,Convertible
...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11910,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11911,Acura,ZDX,premium unleaded (required),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback
11912,Acura,ZDX,premium unleaded (recommended),AUTOMATIC,all wheel drive,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback


In [8]:
df.isna().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3742
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

In [9]:
# Fill multi-hot column missing values
df['Market Category'] = df['Market Category'].fillna("Missing")

In [10]:
np.unique(df['Market Category'].apply(lambda x: x.split(',')).sum())

array(['Crossover', 'Diesel', 'Exotic', 'Factory Tuner', 'Flex Fuel',
       'Hatchback', 'High-Performance', 'Hybrid', 'Luxury', 'Missing',
       'Performance'], dtype='<U16')

In [11]:
def multihot_encode(df, column):
    df = df.copy()
    df[column] = df[column].apply(lambda x: x.split(','))
    all_categories = np.unique(df[column].sum())
    for category in all_categories:
        df[column + '_' + category] = df.apply(lambda x: 1 if category in x[column] else 0, axis=1)
    df = df.drop(column, axis=1)
    return df

In [12]:
# Multi hot encoding
df = multihot_encode(df, column='Market Category')
df

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP,Market Category_Crossover,Market Category_Diesel,Market Category_Exotic,Market Category_Factory Tuner,Market Category_Flex Fuel,Market Category_Hatchback,Market Category_High-Performance,Market Category_Hybrid,Market Category_Luxury,Market Category_Missing,Market Category_Performance
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,26,19,3916,46135,0,0,0,1,0,0,1,0,1,0,0
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,19,3916,40650,0,0,0,0,0,0,0,0,1,0,1
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,20,3916,36350,0,0,0,0,0,0,1,0,1,0,0
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Coupe,28,18,3916,29450,0,0,0,0,0,0,0,0,1,0,1
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Compact,Convertible,28,18,3916,34500,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,46120,1,0,0,0,0,1,0,0,1,0,0
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,56670,1,0,0,0,0,1,0,0,1,0,0
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,50620,1,0,0,0,0,1,0,0,1,0,0
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,Midsize,4dr Hatchback,23,16,204,50920,1,0,0,0,0,1,0,0,1,0,0


In [13]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [14]:
# One-hot encoding
for column in df.select_dtypes('object').columns:
    df = onehot_encode(df, column=column)

In [15]:
df

Year  Engine HP  Engine Cylinders  Number of Doors  highway MPG  \
0      2011      335.0               6.0              2.0           26   
1      2011      300.0               6.0              2.0           28   
2      2011      300.0               6.0              2.0           28   
3      2011      230.0               6.0              2.0           28   
4      2011      230.0               6.0              2.0           28   
...     ...        ...               ...              ...          ...   
11909  2012      300.0               6.0              4.0           23   
11910  2012      300.0               6.0              4.0           23   
11911  2012      300.0               6.0              4.0           23   
11912  2013      300.0               6.0              4.0           23   
11913  2006      221.0               6.0              4.0           26   

       city mpg  Popularity   MSRP  Market Category_Crossover  \
0            19        3916  46135                          0   
1            19        3916  40650                          0   
2            20        3916  36350                          0   
3            18        3916  29450                          0   
4            18        3916  34500                          0   
...         ...         ...    ...                        ...   
11909        16         204  46120                          1   
11910        16         204  56670                          1   
11911        16         204  50620                          1   
11912        16         204  50920                          1   
11913        17          61  28995                          0   

       Market Category_Diesel  Market Category_Exotic  \
0                           0                       0   
1                           0                       0   
2                           0                       0   
3                           0                       0   
4                           0                       0   
...                       ...                     ...   
11909                       0                       0   
11910                       0                       0   
11911                       0                       0   
11912                       0                       0   
11913                       0                       0   

       Market Category_Factory Tuner  Market Category_Flex Fuel  \
0                                  1                          0   
1                                  0                          0   
2                                  0                          0   
3                                  0                          0   
4                                  0                          0   
...                              ...                        ...   
11909                              0                          0   
11910                              0                          0   
11911                              0                          0   
11912                              0                          0   
11913                              0                          0   

       Market Category_Hatchback  Market Category_High-Performance  \
0                              0                                 1   
1                              0                                 0   
2                              0                                 1   
3                              0                                 0   
4                              0                                 0   
...                          ...                               ...   
11909                          1                                 0   
11910                          1                                 0   
11911                          1                                 0   
11912                          1                                 0   
11913                          0                                 0   

      

In [16]:
df.columns[df.isna().sum() > 0] 

Index(['Engine HP', 'Engine Cylinders', 'Number of Doors'], dtype='object')

In [17]:
df['Engine Cylinders'].mode()[0]

4.0

In [18]:
# Fill remaining missing values
df['Engine HP'] = df['Engine HP'].fillna(df['Engine HP'].mean())
for column in ['Engine Cylinders', 'Number of Doors']:
    df[column] = df[column].fillna(df[column].mode()[0])

In [19]:
df.isna().sum().sum()

0

In [20]:
# Split df into X and y
y = df['MSRP']
X = df.drop('MSRP', axis=1)

In [21]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [22]:
len(X_train), len(X_test)

(8339, 3575)

In [23]:
# Scale X
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

### Training

In [24]:
models = {
    "                     Linear Regression": LinearRegression(),
    " Linear Regression (L2 Regularization)": Ridge(),
    " Linear Regression (L1 Regularization)": Lasso(),
    "                   K-Nearest Neighbors": KNeighborsRegressor(),
    "                        Neural Network": MLPRegressor(),
    "Support Vector Machine (Linear Kernel)": LinearSVR(),
    "   Support Vector Machine (RBF Kernel)": SVR(),
    "                         Decision Tree": DecisionTreeRegressor(),
    "                         Random Forest": RandomForestRegressor(),
    "                     Gradient Boosting": GradientBoostingRegressor()
}

In [33]:
for name, model in models.items():
    model.fit(X_train, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.


/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.379e+10, tolerance: 3.024e+09
  model = cd_fast.enet_coordinate_descent(


 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.


/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.


In [34]:
# Without dimensionality reduction
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test, y_test)))

                     Linear Regression R^2 Score: 0.85438
 Linear Regression (L2 Regularization) R^2 Score: 0.85439
 Linear Regression (L1 Regularization) R^2 Score: 0.87139
                   K-Nearest Neighbors R^2 Score: 0.63669
                        Neural Network R^2 Score: 0.60634
Support Vector Machine (Linear Kernel) R^2 Score: -0.26027
   Support Vector Machine (RBF Kernel) R^2 Score: -0.03105
                         Decision Tree R^2 Score: 0.88055
                         Random Forest R^2 Score: 0.88717
                     Gradient Boosting R^2 Score: 0.88115


### Dimensionality Reduction

We have a very high number of columns for our purposes, so let's reduce the data using PCA to have only 100 columns (to increase training speed).

In [26]:
n_components = 100

pca = PCA(n_components=n_components)
pca.fit(X_train)
X_train_reduced = pd.DataFrame(pca.transform(X_train), index=X_train.index, columns=['PC' + str(i) for i in range(1, n_components + 1)])
X_test_reduced = pd.DataFrame(pca.transform(X_test), index=X_test.index, columns=['PC' + str(i) for i in range(1, n_components + 1)])

In [27]:
X_train_reduced

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,PC36,PC37,PC38,PC39,PC40,PC41,PC42,PC43,PC44,PC45,PC46,PC47,PC48,PC49,PC50,PC51,PC52,PC53,PC54,PC55,PC56,PC57,PC58,PC59,PC60,PC61,PC62,PC63,PC64,PC65,PC66,PC67,PC68,PC69,PC70,PC71,PC72,PC73,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,PC83,PC84,PC85,PC86,PC87,PC88,PC89,PC90,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,PC100
10660,-2.338385,-1.961901,-0.683284,0.948398,-0.339212,0.987951,5.554061,2.286027,-0.585908,0.209177,1.784641,-1.465754,0.095816,0.261445,1.397020,0.389584,-2.950083,2.939033,0.464254,-0.773864,-1.656247,-0.125617,0.646752,-0.774746,-2.861910,1.849917,-1.925796,2.303625,-0.170878,-0.707288,1.421008,-0.830326,-0.051818,0.393966,0.177123,-0.685734,0.696558,0.120210,0.618351,-1.342678,0.786018,-0.079224,-0.194720,0.634795,0.144316,-0.100968,-2.914709,-0.884103,-0.042788,0.548707,-2.117412,-4.085974,0.095419,0.168157,5.589218,1.450678,0.242032,-2.152230,-0.643082,8.229124,-1.393479,0.772325,1.206872,-1.887011,2.069705,-0.279108,0.387715,1.128647,1.679061,2.187375,0.924697,-0.439088,-0.050645,-0.044900,-0.187176,-0.682993,-0.716181,-0.441738,-0.687979,0.097165,0.144975,-0.483920,-0.334732,0.656899,-0.060500,-0.168222,0.572675,-0.407501,0.385185,1.065880,0.295335,0.285738,0.617524,0.483821,0.283513,-0.263881,0.185011,-0.683384,-0.309860,0.034132
4140,1.821910,-1.206500,2.117658,0.771862,0.208328,-0.244037,-1.050828,-0.348729,-2.759767,-2.564466,0.338506,-0.264181,0.414628,-2.605665,1.680196,-0.204567,1.661410,-1.781861,1.914039,1.887788,-3.209929,-1.044209,-2.099415,-1.419225,-0.286338,1.656874,-1.229137,-1.597993,-0.506896,-0.867509,0.928987,-2.650841,-0.997711,0.702935,0.504755,-1.072067,-1.851633,0.272347,-0.490061,0.461897,-0.034002,-0.442784,-0.614481,-0.341102,0.245905,0.032939,-0.021717,-0.785569,-0.500027,1.685223,1.004195,-0.641848,-0.447440,3.682710,-0.942459,3.405295,-2.156979,-2.124269,-1.888486,-2.670557,0.360183,0.171233,1.120846,-0.705843,-0.473902,0.403933,-0.550356,-0.725466,0.436600,3.586722,-0.097076,-2.258976,0.413304,-0.945454,0.787970,-0.391461,-1.663269,-0.168760,-0.862323,0.003005,0.614466,0.244339,1.009935,0.158274,0.354934,-1.419481,-0.192989,-1.346031,-0.751620,-0.180131,-0.235520,-1.404001,-0.133072,-2.743340,0.332062,0.312478,0.552718,-0.238475,-1.005225,1.210412
11414,-0.422411,-2.874005,0.068227,0.739230,0.208373,2.880137,-0.997974,2.832223,0.852459,2.403726,0.647396,0.707035,-2.057720,1.936344,-1.038398,4.774394,-0.581190,3.110362,2.765601,-2.417253,0.163445,1.036084,-1.432075,0.731568,-1.799208,-1.006023,3.039201,-0.972880,0.663428,-1.837784,0.223605,-1.913116,0.583421,-1.717809,-1.895999,0.789292,-3.105966,-1.196788,-1.135329,-1.451711,-0.946185,0.518939,0.713544,0.090632,-0.080896,0.098771,-0.358631,0.759724,1.328975,0.300196,0.652819,0.232106,0.828600,1.164734,-0.155896,-0.689781,-1.082455,1.028692,-1.148812,-1.569139,0.278067,-0.172860,-1.064970,-1.045318,-0.414357,-0.001810,-0.122124,-0.102172,0.037770,1.174492,1.180984,-0.504674,0.479196,0.367760,0.619994,-0.136799,-1.308629,0.895532,-0.640914,0.612197,1.295494,-1.027287,0.083126,-0.033322,0.755179,-0.103695,-0.216309,-1.062473,-0.643552,-0.485466,0.501721,1.238448,0.346111,0.698419,-1.297032,-1.404551,0.528711,0.677373,-2.624322,-1.213363
5119,2.812988,2.391385,0.729078,-1.028001,0.784529,2.356440,-1.101400,-1.961816,-1.135974,0.129651,-1.721085,-1.216315,1.384961,0.502612,-0.408801,-1.850856,-1.168009,0.850137,-0.821069,-0.206624,1.718520,-0.117762,-1.023507,-0.825187,-0.598638,-0.399611,0.928681,0.472448,-2.026053,-0.899021,-1.599381,-0.004542,0.349721,0.317779,-1.117015,4.157691,0.121652,-1.291773,1.309994,0.138627,-0.109259,-0.455662,-0.453348,-0.178742,0.080292,-0.063774,0.283364,0.611396,0.302241,1.196655,-0.295686,-2.046877,-0.516537,0.910375,-0.616640,-0.384119,1.356603,-0.443936,-0.193324,0.309048,0.428558,0.734778,0.568956,0.461843,-1.034597,-1.58301

In [28]:
X_train_reduced.var()

PC1      7.233120
PC2      5.739047
PC3      5.330406
PC4      4.258672
PC5      3.511629
           ...   
PC96     1.001375
PC97     1.001327
PC98     1.001301
PC99     1.001277
PC100    1.001221
Length: 100, dtype: float64

In [29]:
for name, model in models.items():
    model.fit(X_train_reduced, y_train)
    print(name + " trained.")

                     Linear Regression trained.
 Linear Regression (L2 Regularization) trained.
 Linear Regression (L1 Regularization) trained.
                   K-Nearest Neighbors trained.


/home/aditya/miniconda3/envs/dsenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


                        Neural Network trained.
Support Vector Machine (Linear Kernel) trained.
   Support Vector Machine (RBF Kernel) trained.
                         Decision Tree trained.
                         Random Forest trained.
                     Gradient Boosting trained.


### Results

In [30]:
# With dimensionality reduction
for name, model in models.items():
    print(name + " R^2 Score: {:.5f}".format(model.score(X_test_reduced, y_test)))

                     Linear Regression R^2 Score: 0.77011
 Linear Regression (L2 Regularization) R^2 Score: 0.77011
 Linear Regression (L1 Regularization) R^2 Score: 0.77012
                   K-Nearest Neighbors R^2 Score: 0.77946
                        Neural Network R^2 Score: 0.52173
Support Vector Machine (Linear Kernel) R^2 Score: -0.25389
   Support Vector Machine (RBF Kernel) R^2 Score: -0.03005
                         Decision Tree R^2 Score: 0.84152
                         Random Forest R^2 Score: 0.84649
                     Gradient Boosting R^2 Score: 0.82749
